In [1]:
from Pipeline import Pipeline
from Language import Language

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from add_transformation import add_transformation
from datasets import load_dataset
# import fugashi
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import libvoikko
import numpy as np
import ast

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christianjensen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
dataset = load_dataset('copenlu/answerable_tydiqa')

Using custom data configuration copenlu--nlp_course_tydiqa-2c79d6e77df16c2a
Reusing dataset parquet (/Users/christianjensen/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-2c79d6e77df16c2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 175.75it/s]


In [5]:
get_data = lambda language: dataset.filter(lambda x: x['language'] == language)

# define languages
languages = {
    'english': Language(
        name = 'english',
        tokenizer = word_tokenize,
        cleaner = lambda text: text.lower(),
        pipeline = Pipeline(get_data('english'))
        ),
    'japanese': Language(
        name = 'japanese',
        tokenizer = lambda text: text.split(), #[word.surface for word in fugashi.Tagger()(text)],
        cleaner = lambda text: text.lower(),
        pipeline = Pipeline(get_data('japanese'))
        ),
    'finnish': Language(
        name = 'finnish',
        tokenizer = word_tokenize,
        cleaner = lambda text: text.lower(),
        pipeline = Pipeline(get_data('finnish'))
        ),
}

Loading cached processed dataset at /Users/christianjensen/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-2c79d6e77df16c2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-966bf95a10f76383.arrow
Loading cached processed dataset at /Users/christianjensen/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-2c79d6e77df16c2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e0b012db28a0a682.arrow
Loading cached processed dataset at /Users/christianjensen/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-2c79d6e77df16c2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-57253105caae3cc1.arrow
Loading cached processed dataset at /Users/christianjensen/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-2c79d6e77df16c2a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-efce6354aa2f51ed.arrow
Load

In [6]:
for language in languages.values():
    language.pipeline.tokenize(language.tokenizer)
    language.pipeline.label_answerable()
    language.pipeline.extract_features()

100%|██████████| 1686/1686 [00:00<00:00, 21959.71ex/s]


In [14]:
# Find the most common first and last words in each language
for language in languages.values():
    count_words = lambda text: np.unique(text, return_counts=True) # Count occurences of words in text
    sort_words = lambda word_count: np.argsort(word_count[1])[::-1] # Get list of sorted indices based on most frequent words
    zip_words = lambda word_counts, sort_indices: list(zip(word_counts[0][sort_indices],word_counts[1][sort_indices])) # Zip the most frequent words with its number of occurences
    def find_most_common(text):
        """Finds the most frequent words in a text together with its number of occurences"""
        word_count = count_words(text)
        return zip_words(word_count, sort_words(word_count))


    tokenized_questions = language.pipeline.train['tokenized_question']
    first = [sublist[0] for sublist in tokenized_questions]
    last = [sublist[-1] for sublist in tokenized_questions]
    
    print(f"""
    Language: {language.name}
    Most frequent first words:
    {find_most_common(first)[:5]}
    Most frequent last words:
    {find_most_common(last)[:5]}
    """)


    Language: english
    Most frequent first word:
    [('When', 2242), ('What', 2101), ('How', 1296), ('Who', 1058), ('Where', 486)]
    Most frequent last word:
    [('?', 7379), ('zombie', 2), ('metabolite', 2), ('\\', 2), ('BCE', 2)]
    

    Language: japanese
    Most frequent first word:
    [('朝比奈', 6), ('三原', 6), ('孫', 6), ('PlayStation', 6), ('加藤', 6)]
    Most frequent last word:
    [('の大きさは？', 4), ('の面積は？', 4), ('はいつ設立した？', 4), ('ＹＡＭＡＨＡがピアノの生産を始めたのはいつ', 2), ('ソ連が崩壊したのはいつ', 2)]
    

    Language: finnish
    Most frequent first word:
    [('Milloin', 3519), ('Mikä', 2328), ('Missä', 1646), ('Kuka', 1619), ('Mitä', 1088)]
    Most frequent last word:
    [('?', 13689), ('tulitaistelussa', 2), ('tohtoriksi+', 2), ('syntynyt', 2), ('pinta-ala', 2)]
    


In [ ]:
for language in languages.values():
    language.pipeline.train(LogisticRegression())
    language.pipeline.validate()